In [2]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import cv2
from sklearn.model_selection import train_test_split
import os
import pickle
from PIL import Image
from resizeimage import resizeimage

In [3]:
device = torch.device('cpu')  # sad reax only

In [4]:
x = []  # image data, start as list
y = []  # output class data, start as list

for folder in os.listdir("./pictures/"):
    for file in os.listdir("./pictures/" + folder + "/"):
        im = cv2.imread("./pictures/" + folder + "/" + file)
        im = np.reshape(im, (3, 256, 144))
        x.append(im)
        #out = np.zeros(9)
        #out[int(folder)] = 1
        out = int(folder)
        y.append(out)

x = np.array(x)
y = np.array(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.002, random_state=0)

In [5]:
batch_size = 32

class TrainDataset(Dataset):
    def __init__(self):
        self.len = x_train.shape[0]
        self.x_data = torch.from_numpy(x_train).type(torch.FloatTensor)
        self.y_data = torch.from_numpy(y_train).type(torch.LongTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [6]:
class TestDataset(Dataset):
    def __init__(self):
        self.len = x_test.shape[0]
        self.x_data = torch.from_numpy(x_test).type(torch.FloatTensor)
        self.y_data = torch.from_numpy(y_test).type(torch.LongTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 10, kernel_size=5)
        self.conv3 = nn.Conv2d(10, 10, kernel_size=5)
        self.conv4 = nn.Conv2d(10, 10, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(600, 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 9)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = F.relu(self.mp(self.conv3(x)))
        x = F.relu(self.mp(self.conv4(x)))
        #x = F.relu(self.mp(self.conv5(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [8]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


In [9]:
def train(epoch, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader, 0):
        data, target = Variable(data), Variable(target)
        #print(data)
        #print(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(epoch, batch_idx, loss.item())

In [10]:
if __name__ == '__main__':
    train_dataset = TrainDataset()
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    """for epoch in range(5):
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = Variable(inputs), Variable(labels)
            print(epoch, i, "inputs", inputs.data, "labels", labels.data)"""

    for epoch in range(1, 4):
        train(epoch, train_loader)

        # test(test_loader)

1 0 2.5828590393066406
1 10 2.1726722717285156
1 20 2.3776512145996094
1 30 2.166435956954956
1 40 2.1244795322418213
1 50 2.1663758754730225
1 60 2.1194489002227783
1 70 2.088878870010376
1 80 2.1738667488098145
1 90 1.7616313695907593
1 100 1.7947118282318115
1 110 1.1493566036224365
1 120 1.2945349216461182
1 130 1.0034338235855103
1 140 0.9169489741325378
1 150 1.1350507736206055
1 160 1.3008159399032593
1 170 0.9950670003890991
1 180 0.9966185092926025
1 190 0.8571208715438843
1 200 1.0186917781829834
1 210 0.5110820531845093
1 220 0.7313164472579956
1 230 0.7766296863555908
1 240 0.7412429451942444
1 250 0.5733363628387451
2 0 0.41341909766197205
2 10 0.5041103363037109
2 20 0.7720800638198853
2 30 0.790165901184082
2 40 0.3904699683189392
2 50 0.5660865902900696
2 60 0.7023823261260986
2 70 0.586706280708313
2 80 0.35064148902893066
2 90 0.4769468903541565
2 100 0.6049098372459412
2 110 0.36982154846191406
2 120 0.30825063586235046
2 130 0.34340381622314453
2 140 0.4575759470462

In [ ]:
feedback = ["Move down and right", "Move down", "Move down and left", "Move right", "Perfect!", "Move left", "Move up and right", "Move up", "Move right"]
while True:
    for file in os.listdir("../../../../Downloads/"):
        l = []
        if 'ss_file' in file:
            print("here")
            try:
                with open("../../../../Downloads/" + file, 'r+b') as f:
                    with Image.open(f) as image:
                        resized_image = resizeimage.resize_cover(image, [256, 144])
                        resized_image.save("../../../../Downloads/" + file, image.format)
                    image.close()
                f.close()
                im = cv2.imread("../../../../Downloads/" + file)
                im = np.reshape(im, (1, 3, 256, 144))
                
                x = np.array(im)
                im = im.tolist()
                # %store im >> ../../../../../natha/thingy/htdocs/hophacks/data.txt
                x = torch.from_numpy(x).type(torch.FloatTensor)
                output = model(x)
                try:
                    os.remove("../../../../../natha/thingy/htdocs/hophacks/file.txt")
                except:
                    pass
                output = output.data.numpy().tolist()
                %store output >> ../../../../../natha/thingy/htdocs/hophacks/data.txt
                output = output[0]
                output = output.index(max(output))
                %store feedback[output] >> ../../../../../natha/thingy/htdocs/hophacks/file.txt
                try:
                    os.remove("../../../../Downloads/" + file)
                except:
                    pass
            except:
                pass

here
Writing 'output' (list) to file '../../../../../natha/thingy/htdocs/hophacks/data.txt'.
Writing 'feedback[output]' (str) to file '../../../../../natha/thingy/htdocs/hophacks/file.txt'.
here
Writing 'output' (list) to file '../../../../../natha/thingy/htdocs/hophacks/data.txt'.
Writing 'feedback[output]' (str) to file '../../../../../natha/thingy/htdocs/hophacks/file.txt'.
here
Writing 'output' (list) to file '../../../../../natha/thingy/htdocs/hophacks/data.txt'.
Writing 'feedback[output]' (str) to file '../../../../../natha/thingy/htdocs/hophacks/file.txt'.
here
Writing 'output' (list) to file '../../../../../natha/thingy/htdocs/hophacks/data.txt'.
Writing 'feedback[output]' (str) to file '../../../../../natha/thingy/htdocs/hophacks/file.txt'.
here
Writing 'output' (list) to file '../../../../../natha/thingy/htdocs/hophacks/data.txt'.
Writing 'feedback[output]' (str) to file '../../../../../natha/thingy/htdocs/hophacks/file.txt'.
here
Writing 'output' (list) to file '../../../../